# Experiments

In [99]:
import pyvista as pv
import numpy as np
import matplotlib.pyplot as plt
import vtk as vtk

In [100]:
%matplotlib widget

# Load the case with `vtkOpenFOAMReader`

In [101]:
def createTimeList(array): # T the list we want to create and array the vtkDoubleArray containing the time values
    T = []
    for i in range(0, array.GetMaxId()+1):
        T.append(array.GetValue(i))
    return T

In [102]:
reader = vtk.vtkOpenFOAMReader()
reader.SetFileName("C:/Users/pret/Desktop/INSA HDF 5A/PLP 5A/OpenFOAM_example_case/case.foam")
reader.UpdateInformation()
time = reader.GetTimeValues() # All the time values of the simulation in the form of a vtkDoubleArray
reader.SetTimeValue(time.GetValue(2)) # Set time value to whichever one we want to visualize 
reader.Update()
multi_block = pv.wrap(reader.GetOutput()) 
print(time) 

vtkDoubleArray (00000287ED6A9880)
  Debug: Off
  Modified Time: 1761291
  Reference Count: 2
  Registered Events: (none)
  Name: (none)
  Data type: double
  Size: 3
  MaxId: 2
  NumberOfComponents: 1
  Information: 0000000000000000
  Name: (none)
  Number Of Components: 1
  Number Of Tuples: 3
  Size: 3
  MaxId: 2
  LookupTable: (none)




In [103]:
T = createTimeList(time) # This gives us a list of all the time values
print(T)

[0.0, 500.0, 1000.0]


In [104]:
mesh = multi_block[0]
mesh

UnstructuredGrid (0x287edb71280)
  N Cells:	12225
  N Points:	25012
  X Bounds:	-2.060e-02, 2.900e-01
  Y Bounds:	-2.540e-02, 2.540e-02
  Z Bounds:	-5.000e-04, 5.000e-04
  N Arrays:	10

We can notice that we have the same fields twice. 
Each field is stored as both point data and cell data, this means that we have 10 N arrays instead of 5.

Let's create two datasets: one that only contains the cell data and one that only contains the point data.

In [105]:
# This dataset will only contain cell data
multi_block = pv.wrap(reader.GetOutput()) 
mesh_cell = multi_block[0]
mesh_cell.clear_point_data()
mesh_cell

UnstructuredGrid (0x287edb719a0)
  N Cells:	12225
  N Points:	25012
  X Bounds:	-2.060e-02, 2.900e-01
  Y Bounds:	-2.540e-02, 2.540e-02
  Z Bounds:	-5.000e-04, 5.000e-04
  N Arrays:	5

In [106]:
# This dataset will only contain point data
multi_block = pv.wrap(reader.GetOutput()) 
mesh_point = multi_block[0]
mesh_point.clear_cell_data()
mesh_point

UnstructuredGrid (0x287edb696a0)
  N Cells:	12225
  N Points:	25012
  X Bounds:	-2.060e-02, 2.900e-01
  Y Bounds:	-2.540e-02, 2.540e-02
  Z Bounds:	-5.000e-04, 5.000e-04
  N Arrays:	5

In [5]:
# n = reader.GetNumberOfPatchArrays()
# reader.SetPatchArrayStatus(reader.GetPatchArrayName(0), 1) # 0 for disabled, 1 for enabled 

## Show the mesh 

In [107]:
# Create a plotting object to display vtk meshes or numpy arrays
p = pv.Plotter()
# Show the mesh in wireframe style
p.add_mesh(mesh, style="wireframe", color="w")
# View in the xy plane
p.view_xy()
# Show the axes
p.add_axes()
# Display the plotting window
p.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

## Plot the domain with a scalar field


In [108]:
# Plot the domain based on the cell data
p = pv.Plotter()
p.add_mesh(mesh_cell, scalars='U')
p.view_xy()
p.add_axes()
p.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [109]:
mesh_cell

UnstructuredGrid (0x287edb719a0)
  N Cells:	12225
  N Points:	25012
  X Bounds:	-2.060e-02, 2.900e-01
  Y Bounds:	-2.540e-02, 2.540e-02
  Z Bounds:	-5.000e-04, 5.000e-04
  N Arrays:	6

In [110]:
# Plot the domain based on the point data
p = pv.Plotter()
p.add_mesh(mesh_point, scalars='U')
p.view_xy()
p.add_axes()
p.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [111]:
mesh_point

UnstructuredGrid (0x287edb696a0)
  N Cells:	12225
  N Points:	25012
  X Bounds:	-2.060e-02, 2.900e-01
  Y Bounds:	-2.540e-02, 2.540e-02
  Z Bounds:	-5.000e-04, 5.000e-04
  N Arrays:	6

### How to access the vector components

In [112]:
mesh_cell.cell_data

pyvista DataSetAttributes
Association     : CELL
Active Scalars  : U-normed
Active Vectors  : U
Active Texture  : None
Active Normals  : None
Contains arrays :
    U                       float32  (12225, 3)           VECTORS
    epsilon                 float32  (12225,)
    k                       float32  (12225,)
    nut                     float32  (12225,)
    p                       float32  (12225,)
    U-normed                float32  (12225,)             SCALARS

In [113]:
ux = mesh_cell['U'][:,0]
uy = mesh_cell['U'][:,1]
uz = mesh_cell['U'][:,2]

mesh_cell.cell_data["ux"] = ux
mesh_cell.cell_data["uy"] = uy
mesh_cell.cell_data["uz"] = uz

mesh_cell

UnstructuredGrid (0x287edb719a0)
  N Cells:	12225
  N Points:	25012
  X Bounds:	-2.060e-02, 2.900e-01
  Y Bounds:	-2.540e-02, 2.540e-02
  Z Bounds:	-5.000e-04, 5.000e-04
  N Arrays:	9

In [114]:
mesh_point.point_data

pyvista DataSetAttributes
Association     : POINT
Active Scalars  : U-normed
Active Vectors  : U
Active Texture  : None
Active Normals  : None
Contains arrays :
    U                       float32  (25012, 3)           VECTORS
    epsilon                 float32  (25012,)
    k                       float32  (25012,)
    nut                     float32  (25012,)
    p                       float32  (25012,)
    U-normed                float32  (25012,)             SCALARS

In [115]:
ux = mesh_point['U'][:,0]
uy = mesh_point['U'][:,1]
uz = mesh_point['U'][:,2]

mesh_point.point_data["ux"] = ux
mesh_point.point_data["uy"] = uy
mesh_point.point_data["uz"] = uz

mesh_point

UnstructuredGrid (0x287edb696a0)
  N Cells:	12225
  N Points:	25012
  X Bounds:	-2.060e-02, 2.900e-01
  Y Bounds:	-2.540e-02, 2.540e-02
  Z Bounds:	-5.000e-04, 5.000e-04
  N Arrays:	9

## Let's plot the streamlines

In [116]:
X_center = (mesh.bounds[0]+mesh.bounds[1])/2
Y_center = (mesh.bounds[2]+mesh.bounds[3])/2
Z_center = (mesh.bounds[4]+mesh.bounds[5])/2
centers = [X_center, Y_center, Z_center] 


face = mesh.slice(normal='x') # "normal" should be set as the direction of the flow
slices = mesh.slice_along_axis(n=8, axis="x") # axis is the axis of the flow
faces = slices.combine() # converts Multiblock into UnstructuredGrid 

str1 = mesh.streamlines_from_source(faces, vectors='U') # streamlines using multiple surfaces
str2 = mesh.streamlines_from_source(face, vectors='U') # streamlines using a singular surface
# both require a dataset for the surface

str3 = mesh.streamlines( 
n_points=100,
vectors='U',
pointa=(mesh.bounds[0], mesh.bounds[2], centers[2]),
pointb=(mesh.bounds[0], mesh.bounds[3], centers[2])
)  # Streamlines using a line

We'll compare between the streamlines obtained from one singular surface (str1) and those obtained using a higher number of equidistant parallel surfaces (str2). 
The most accurate of the two cases is the one that resembles what we obtain with ParaView the most.

In [117]:
# Streamlines obtained from a higher number of equidistant parallel surfaces
p = pv.Plotter()
p.add_mesh(mesh, color="w")
p.add_mesh(str1.tube(radius=0.0004))
# View in the xy plane
p.view_xy()
# Show the axes
p.add_axes()
# Display the plotting window
p.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [118]:
# Streamlines obtained from one singular surface
p = pv.Plotter()
p.add_mesh(mesh, color="w")
p.add_mesh(str2.tube(radius=0.0004))
# View in the xy plane
p.view_xy()
# Show the axes
p.add_axes()
# Display the plotting window
p.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

## Let's determine the friction velocity

In [9]:
wall = mesh.slice(normal='y', origin=(X_center, mesh.bounds[3], 0))
wall2 = mesh.slice(normal='-y', origin=(X_center, mesh.bounds[2], 0))
wall3 = mesh.slice(normal='-y', origin=(X_center, 0, 0))
p = pv.Plotter()
p.add_mesh(wall)
p.add_mesh(wall2)
p.add_mesh(wall3)
p.view_xy()
p.add_axes()
p.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [119]:
def gradients_to_dict(arr):
    keys = np.array(["dux/dx", "dux/dy", "dux/dz", "duy/dx", "duy/dy", "duy/dz", "duz/dx", "duz/dy", "duz/dz"])
    keys = keys.reshape((3,3))[:,:arr.shape[1]].ravel()
    return dict(zip(keys, arr.T))

def gradients_to_dict2(arr):
    keys = np.array(["dU/dx", "dU/dy", "dU/dz"])
    keys = keys.reshape((3,1))[:,:arr.shape[1]].ravel()
    return dict(zip(keys, arr.T))

In [120]:
# deriv will contain the data from the gradient of U (9 components)
deriv_cell = mesh_cell.compute_derivative(scalars="U")
# deriv2 will contain the data from the gradient of U-normed (3 components)
deriv2_cell = mesh_cell.compute_derivative(scalars="U-normed")

grad_cell = gradients_to_dict(deriv_cell["gradient"])
grad2_cell = gradients_to_dict2(deriv2_cell["gradient"])

mesh_cell.cell_data["dU/dx"] = grad2_cell["dU/dx"]
mesh_cell.cell_data["dU/dy"] = grad2_cell["dU/dy"]
mesh_cell.cell_data["dU/dz"] = grad2_cell["dU/dz"]
mesh_cell.cell_data["dux/dx"] = grad_cell["dux/dx"]
mesh_cell.cell_data["dux/dy"] = grad_cell["dux/dy"]
mesh_cell.cell_data["dux/dz"] = grad_cell["dux/dz"]
mesh_cell.cell_data["duy/dx"] = grad_cell["duy/dx"]
mesh_cell.cell_data["duy/dy"] = grad_cell["duy/dy"]
mesh_cell.cell_data["duy/dz"] = grad_cell["duy/dz"]
mesh_cell.cell_data["duz/dx"] = grad_cell["duz/dx"]
mesh_cell.cell_data["duz/dy"] = grad_cell["duz/dy"]
mesh_cell.cell_data["duz/dz"] = grad_cell["duz/dz"]

mesh_cell

UnstructuredGrid (0x287edb719a0)
  N Cells:	12225
  N Points:	25012
  X Bounds:	-2.060e-02, 2.900e-01
  Y Bounds:	-2.540e-02, 2.540e-02
  Z Bounds:	-5.000e-04, 5.000e-04
  N Arrays:	21

In [121]:
# deriv will contain the data from the gradient of U (9 components)
deriv_point = mesh_point.compute_derivative(scalars="U")
# deriv2 will contain the data from the gradient of U-normed (3 components)
deriv2_point = mesh_point.compute_derivative(scalars="U-normed")

grad_point = gradients_to_dict(deriv_point["gradient"])
grad2_point = gradients_to_dict2(deriv2_point["gradient"])

mesh_point.point_data["dU/dx"] = grad2_point["dU/dx"]
mesh_point.point_data["dU/dy"] = grad2_point["dU/dy"]
mesh_point.point_data["dU/dz"] = grad2_point["dU/dz"]
mesh_point.point_data["dux/dx"] = grad_point["dux/dx"]
mesh_point.point_data["dux/dy"] = grad_point["dux/dy"]
mesh_point.point_data["dux/dz"] = grad_point["dux/dz"]
mesh_point.point_data["duy/dx"] = grad_point["duy/dx"]
mesh_point.point_data["duy/dy"] = grad_point["duy/dy"]
mesh_point.point_data["duy/dz"] = grad_point["duy/dz"]
mesh_point.point_data["duz/dx"] = grad_point["duz/dx"]
mesh_point.point_data["duz/dy"] = grad_point["duz/dy"]
mesh_point.point_data["duz/dz"] = grad_point["duz/dz"]

mesh_point

UnstructuredGrid (0x287edb696a0)
  N Cells:	12225
  N Points:	25012
  X Bounds:	-2.060e-02, 2.900e-01
  Y Bounds:	-2.540e-02, 2.540e-02
  Z Bounds:	-5.000e-04, 5.000e-04
  N Arrays:	21

In [81]:
p = pv.Plotter()
p.add_mesh(mesh_cell, color="w", scalars='dU/dy') 
# View in the xy plane
p.view_xy()
# Show the axes
p.add_axes()
# Display the plotting window
p.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)